In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
# this model have accuracy of 100% and loss of 0.0015
model = load_model('eye_model_by_saurabey.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 50


# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device
eyes_detected = False

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    eyes = detect_eyes(frame)
    

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)
        
        eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to RGB format
        resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2RGB)

        # Convert the ROI to numpy array
        resized_roi = np.asarray(resized_roi)

        # Expand dimensions to match the input shape of your CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi / 255.0

        # Pass the preprocessed ROI through your CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)
        # print(predicted_class)
        # frame = cv2.rectangle(frame, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
        
        eyes_detected = True
    else:
        eyes_detected = False
        
    frame = cv2.flip(frame, 1)

    if eyes_detected:
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 30ms/step


In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import random



# Load the pre-trained CNN model
model = load_model('latest3_rgb.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle', 'middle']
folders_number = [6, 8, 7, 0, 2, 1, 3, 5, 4]

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes

# Define the desired dimensions
desired_width = 250
desired_height = 100

# Create the black window
black_window = np.zeros((700, 1400, 3), dtype=np.uint8)
rectangle_center = [[103, 53], [633, 53], [1163, 53], [103, 343], [633, 343], [1163, 343], [103, 633], [633, 633], [1163, 633]]

green_rectangle_coordinates = []
for i in range(2):
    for j in range(2):
        green_rectangle_coordinates.append([3+j*530, 3+i*290])
        # rectangle_center.append([3+j*530+100, 3+i*290+50])

print(rectangle_center)
# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    cv2.imshow('Game', black_window)

    eyes = detect_eyes(frame)
    

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        # eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        # eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)
        
        if eye_x == eye_1x:
            left_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
            right_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
        else:
            left_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
            right_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
        
        # Resize the ROI to match the desired dimensions
        resized_left_eye_roi = cv2.resize(left_eye_roi, (125, 100))
        resized_right_eye_roi = cv2.resize(right_eye_roi, (125, 100))
        
        combined_image = cv2.hconcat([resized_left_eye_roi, resized_right_eye_roi])

        # eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        # resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to grayscale
        resized_roi = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)

        # Expand dimensions to match the input shape of the CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi.astype('float32') / 255

        # Pass the preprocessed ROI through the CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)

        # Get the real class number
        real_class = folders_number[predicted_class[0]]
        
        selected_coordinates = rectangle_center[real_class]
        # Remove the selected coordinates from the list
        for i in green_rectangle_coordinates:
            if abs(selected_coordinates[0]- i[0]) <= 100 and abs(selected_coordinates[1]- i[1]) <= 50:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                break
            
            elif abs(selected_coordinates[0]- i[0]) <= 125 and abs(selected_coordinates[1]- i[1]) <= 63:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                break
                
            
            elif abs(selected_coordinates[0]- i[0]) <= 150 and abs(selected_coordinates[1]- i[1]) <= 75:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                break
                
            
            elif abs(selected_coordinates[0]- i[0]) <= 175 and abs(selected_coordinates[1]- i[1]) <= 83:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                break
                
                
            elif abs(selected_coordinates[0]- i[0]) <= 200 and abs(selected_coordinates[1]- i[1]) <= 100:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                
                break
            
            elif abs(selected_coordinates[0]- i[0]) <=250  and abs(selected_coordinates[1]- i[1]) <= 125:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                
                break
            
            elif abs(selected_coordinates[0]- i[0]) <=300  and abs(selected_coordinates[1]- i[1]) <= 150:
                green_rectangle_coordinates.remove(i)
                green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                break



        for (x,y,w,h) in eyes:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    # if len(green_rectangle_coordinates) < 9:
    #     green_rectangle_coordinates.append([random.randint(0, 1593), random.randint(0, 873)])
        

    
    # Clear the black window
    black_window.fill(0)

    for [x, y] in green_rectangle_coordinates:
        cv2.rectangle(black_window, (x, y), (x+200, y+100), (0, 255, 0), cv2.FILLED)
        
        

    cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


[[103, 53], [633, 53], [1163, 53], [103, 343], [633, 343], [1163, 343], [103, 633], [633, 633], [1163, 633]]
1/1 [==============================] - 0s 38ms/step


In [ ]:
# # flip all the images in the folder bottom_left
# import os
# import cv2
# list1 = ['bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']
# for folder in list1:
#     folder_path = folder
#     filenames = os.listdir(folder_path)

#     for filename in filenames:
#         image = cv2.imread(os.path.join(folder_path, filename))
#         image = cv2.flip(image, 1)
#         cv2.imwrite(os.path.join(folder_path, filename), image)

    


In [ ]:
import cv2
import numpy as np


def mouse_callback(event, x, y, flags, param):
    global rect_origin

    if event == cv2.EVENT_LBUTTONDOWN:
        rect_origin = (x, y)

# Create a black window
window = np.zeros((500, 500, 3), np.uint8)
cv2.namedWindow("Rectangle Window")
cv2.setMouseCallback("Rectangle Window", mouse_callback)

rect_origin = None

while True:
    # Display the window
    cv2.imshow("Rectangle Window", window)

    # Check for key press
    key = cv2.waitKey(1) & 0xFF

    # If 'q' is pressed, exit the loop
    if key == ord('q'):
        break

    # If rectangle origin is set
    if rect_origin is not None:
        # Get the current mouse position
        current_pos = np.array((cv2.waitKey(1) & 0xFF, cv2.waitKey(1) & 0xFF))

        # Calculate rectangle coordinates
        x1 = min(rect_origin[0], current_pos[0])
        y1 = min(rect_origin[1], current_pos[1])
        x2 = max(rect_origin[0], current_pos[0])
        y2 = max(rect_origin[1], current_pos[1])

        # Draw the rectangle on the window
        window = np.zeros((500, 500, 3), np.uint8)  # Clear the window
        cv2.rectangle(window, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Close all windows
cv2.destroyAllWindows()


In [ ]:
import random
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()
# Define the list of picture coordinates
picture_coordinates = [(100, 100), (200, 200), (300, 300), (400, 400), (500, 500)]
picture_coordinates
points = 0

# Randomly select initial coordinates for each picture
random.shuffle(picture_coordinates)

# take the window size
window_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
x1 = int(window_size[0]/2) - 100
y1 = int(window_size[1]/2) - 50
x2 = int(window_size[0]/2) + 100
y2 = int(window_size[1]/2) + 50


text = 'Rectangle Game'
text_size= cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

touching_rectangle = False
touching = False

x1_black = 300
y1_black = 100
x2_black = 500
y2_black = 200

start_game = False


# All the coordinates related to the snake Game
# Game constants
WINDOW_WIDTH = window_size[0]
WINDOW_HEIGHT = window_size[1]
SNAKE_SIZE = 20
FOOD_SIZE = 20

snake_x_player1 = [WINDOW_WIDTH // 4]
snake_y_player1 = [WINDOW_HEIGHT // 2]
food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)

# Initialize player 2 snake and food positions
snake_x_player2 = [3 * WINDOW_WIDTH // 4]
snake_y_player2 = [WINDOW_HEIGHT // 2]

score_player1 = 0
score_player2 = 0
snake_game = False

eye_control = False





# for the eye control

# Load the pre-trained CNN model
model = load_model('babal.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle', 'middle']
folders_number = [6, 8, 7, 0, 2, 1, 3, 5, 4]

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes

# Define the desired dimensions
desired_width = 250
desired_height = 50

# Create the black window
black_window = np.zeros((700, 1400, 3), dtype=np.uint8)
rectangle_center = [[103, 53], [633, 53], [1163, 53], [103, 343], [633, 343], [1163, 343], [103, 633], [633, 633], [1163, 633]]

green_rectangle_coordinates = []
for i in range(2):
    for j in range(2):
        green_rectangle_coordinates.append([3+j*530, 3+i*290])

reset = False


# [1522, 135], [867, 799], [1473, 432], [853, 802]]
thumb_index_middle_press = 0




while True:
    success, image = cap.read()

    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # Here the image arrray is read only mode
    image.flags.writeable = False
    results = hands.process(image)
    # print(results)
    # Here the image arrray is writeable mode
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks and eye_control == False:
        for hand_landmarks in results.multi_hand_landmarks:
            

            # Check if index finger is touching any picture
            index_finger_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image.shape[0]
            )
            thumb_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image.shape[0]
            )
            middle_finger_coordinates = (
                hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image.shape[1],
                hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image.shape[0]
            )
            if abs(index_finger_coordinates[0] - thumb_coordinates[0]) < 15 and abs(index_finger_coordinates[1] - thumb_coordinates[1]) < 15:
                if start_game == False:
                    if abs(x1 - index_finger_coordinates[0]+ 100) < 100 and abs(y1 - index_finger_coordinates[1]+50) < 50 :
                        
                        print("Touching the rectangle")
                        touching_rectangle = True
                        start_game = True
                    
                    elif abs(x1 - index_finger_coordinates[0]+ 100) < 100 and abs(100 - index_finger_coordinates[1]+50) < 50 :
                        print('This is awesome')
                        snake_game = True
                        start_game = True
                        
                    elif abs(x1 - index_finger_coordinates[0]+ 100) < 100 and abs(y2+30 - index_finger_coordinates[1]+50) < 50 :
                        print('This is awesome')
                        eye_control = True
                        start_game = True
                
                elif abs(middle_finger_coordinates[0] - index_finger_coordinates[0]) < 20 and abs(middle_finger_coordinates[1] - index_finger_coordinates[1]) < 20:
                    thumb_index_middle_press += 1
                    if thumb_index_middle_press % 3 ==0:
                        print("This is awesome")
                        # touching_rectangle==False 
                        start_game = False
                        snake_game = False
                        touching_rectangle = False
                        touching = False
                    
                        
                
                
                elif abs(x1_black - index_finger_coordinates[0]+100) < 100 and abs(y1_black - index_finger_coordinates[1]+50) < 50:
                    
                    touching = True
                    print("Touching the black rectangle")
                    

                for i, (x,y) in enumerate(picture_coordinates):
                    if abs(x - index_finger_coordinates[0]) < 30 and abs(y - index_finger_coordinates[1]) < 30:
                    
                    # Index finger is touching the picture
                        picture_coordinates.pop(i)
                        picture_coordinates.append((random.randint(0, window_size[0]), random.randint(0, window_size[1])))
                        points += 1
                        break
            
            
            else:
                touching = False
                
                num_hands = len(results.multi_hand_landmarks)
        if  len(results.multi_hand_landmarks) >= 2 and snake_game:
            player1_landmarks = results.multi_hand_landmarks[0]
            player2_landmarks = results.multi_hand_landmarks[1]
            
                        # Player 1 - Get index finger tip coordinates
            index_finger_x_player1 = int(player1_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * WINDOW_WIDTH)
            index_finger_y_player1 = int(player1_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * WINDOW_HEIGHT)

            # Player 2 - Get index finger tip coordinates
            index_finger_x_player2 = int(player2_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * WINDOW_WIDTH)
            index_finger_y_player2 = int(player2_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * WINDOW_HEIGHT)

            # Player 1 - Move snake based on index finger direction
            if len(snake_x_player1) > 1:
                dx = snake_x_player1[-1] - snake_x_player1[-2]
                dy = snake_y_player1[-1] - snake_y_player1[-2]
                
                # asari ni lekna mildo raixa
                # if abs(dx) > abs(dy):
                #     index_finger_x_player1 += SNAKE_SIZE if dx > 0 else -SNAKE_SIZE
                # else:
                #     index_finger_y_player1 += SNAKE_SIZE if dy > 0 else -SNAKE_SIZE
                
                if abs(dx) > abs(dy):
                # If dx > 0, it means the movement is towards the right
                    if dx > 0:
                        index_finger_x_player1 += SNAKE_SIZE
                    else:
                        index_finger_x_player1 -= SNAKE_SIZE
                else:
                    # If dy > 0, it means the movement is downwards
                    if dy > 0:
                        index_finger_y_player1 += SNAKE_SIZE
                    else:
                        index_finger_y_player1 -= SNAKE_SIZE


                    

            # Player 2 - Move snake based on index finger direction
            if len(snake_x_player2) > 1:
                dx = snake_x_player2[-1] - snake_x_player2[-2]
                dy = snake_y_player2[-1] - snake_y_player2[-2]

                if abs(dx) > abs(dy):
                    index_finger_x_player2 += SNAKE_SIZE if dx > 0 else -SNAKE_SIZE
                else:
                    index_finger_y_player2 += SNAKE_SIZE if dy > 0 else -SNAKE_SIZE

            # Player 1 - Check collision with food
            if abs(snake_x_player1[-1] - food_x) < SNAKE_SIZE and abs(snake_y_player1[-1] - food_y) < SNAKE_SIZE:
                snake_x_player1.append(food_x)
                snake_y_player1.append(food_y)
                food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
                food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)
                score_player1 += 1

            # Player 2 - Check collision with food
            if abs(snake_x_player2[-1] - food_x) < SNAKE_SIZE and abs(snake_y_player2[-1] - food_y) < SNAKE_SIZE:
                snake_x_player2.append(food_x)
                snake_y_player2.append(food_y)
                food_x = random.randint(0, WINDOW_WIDTH - FOOD_SIZE)
                food_y = random.randint(0, WINDOW_HEIGHT - FOOD_SIZE)
                score_player2 += 1

            # Player 1 - Move snake
            snake_x_player1.append(index_finger_x_player1)
            snake_y_player1.append(index_finger_y_player1)

            # Player 2 - Move snake
            snake_x_player2.append(index_finger_x_player2)
            snake_y_player2.append(index_finger_y_player2)


            # Comment out the Remove tail segment to make writting place.
            # # Player 1 - Remove tail segment
            if len(snake_x_player1) > 1:
                snake_x_player1.pop(0)
                snake_y_player1.pop(0)

            # Player 2 - Remove tail segment
            if len(snake_x_player2) > 1:
                snake_x_player2.pop(0)
                snake_y_player2.pop(0)

            # Player 1 - Check collision with boundaries
            if (
                snake_x_player1[-1] < 0 or
                snake_x_player1[-1] >= WINDOW_WIDTH or
                snake_y_player1[-1] < 0 or
                snake_y_player1[-1] >= WINDOW_HEIGHT
            ):
                # Game over for player 1
                cv2.putText(image, "Game Over - Player 1", (WINDOW_WIDTH // 2 - 100, WINDOW_HEIGHT // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                snake_x_player1 = [WINDOW_WIDTH // 4]
                snake_y_player1 = [WINDOW_HEIGHT // 2]
                score_player1 = 0

            # Player 2 - Check collision with boundaries
            if (
                snake_x_player2[-1] < 0 or
                snake_x_player2[-1] >= WINDOW_WIDTH or
                snake_y_player2[-1] < 0 or
                snake_y_player2[-1] >= WINDOW_HEIGHT
            ):
                # Game over for player 2
                cv2.putText(image, "Game Over - Player 2", (WINDOW_WIDTH // 2 - 100, WINDOW_HEIGHT // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                snake_x_player2 = [3 * WINDOW_WIDTH // 4]
                snake_y_player2 = [WINDOW_HEIGHT // 2]
                score_player2 = 0

            # Draw player 1 snake
            for x, y in zip(snake_x_player1, snake_y_player1):
                cv2.rectangle(image, (x, y), (x + SNAKE_SIZE, y + SNAKE_SIZE), (0, 255, 0), cv2.FILLED)

            # Draw player 2 snake
            for x, y in zip(snake_x_player2, snake_y_player2):
                cv2.rectangle(image, (x, y), (x + SNAKE_SIZE, y + SNAKE_SIZE), (255, 0, 0), cv2.FILLED)

            # Draw food
            cv2.rectangle(image, (food_x, food_y), (food_x + FOOD_SIZE, food_y + FOOD_SIZE), (0, 0, 255), cv2.FILLED)

    if snake_game:
        # Draw scores
        cv2.putText(image, f"Player 1 Score: {score_player1}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(image, f"Player 2 Score: {score_player2}", (WINDOW_WIDTH - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            
    if touching and snake_game == False:
            # Calculate the new position of the black box based on the index finger's movement
            x1_black = int(index_finger_coordinates[0]) - 100
            y1_black = int(index_finger_coordinates[1]) - 50
            x2_black = int(index_finger_coordinates[0]) + 100
            y2_black = int(index_finger_coordinates[1]) + 50
                
    if touching_rectangle and snake_game == False:
        

        cv2.rectangle(image, (x1_black, y1_black), (x2_black, y2_black), (0,0,0), cv2.FILLED)
        
        
        for x, y in picture_coordinates:
           
            cv2.rectangle(image, (x-30, y-30), (x+30, y+30), (0, 255, 0), cv2.FILLED)
        cv2.putText(image, f"Points: {points}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    if eye_control:
        cv2.imshow('Game', black_window)
        eyes = detect_eyes(image)
        
        if len(eyes) >= 2:
            (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
            eye_x = min(eye_1x, eye_2x)
            eye_y = min(eye_1y, eye_2y)
            eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
            eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)

            eye_roi = image[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

            # Resize the ROI to match the desired dimensions
            resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

            # Convert the ROI to grayscale
            resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2GRAY)

            # Expand dimensions to match the input shape of the CNN model
            resized_roi = np.expand_dims(resized_roi, axis=0)

            # Normalize pixel values
            resized_roi = resized_roi.astype('float32') / 255

            # Pass the preprocessed ROI through the CNN model for prediction
            predictions = model.predict(resized_roi)
            predicted_class = np.argmax(predictions, axis=1)

            # Get the real class number
            real_class = folders_number[predicted_class[0]]
            
            selected_coordinates = rectangle_center[real_class]
            # Remove the selected coordinates from the list
            for i in green_rectangle_coordinates:
                if abs(selected_coordinates[0]- i[0]) <= 100 and abs(selected_coordinates[1]- i[1]) <= 50:
                    green_rectangle_coordinates.remove(i)
                    
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])

                    break
                elif abs(selected_coordinates[0]- i[0]) <= 125 and abs(selected_coordinates[1]- i[1]) <= 63:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    break
                    
                
                elif abs(selected_coordinates[0]- i[0]) <= 150 and abs(selected_coordinates[1]- i[1]) <= 75:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    break
                    
                
                elif abs(selected_coordinates[0]- i[0]) <= 175 and abs(selected_coordinates[1]- i[1]) <= 83:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    break
                    
                    
                elif abs(selected_coordinates[0]- i[0]) <= 200 and abs(selected_coordinates[1]- i[1]) <= 100:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    
                    break
                
                elif abs(selected_coordinates[0]- i[0]) <=250  and abs(selected_coordinates[1]- i[1]) <= 125:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    
                    break
                
                elif abs(selected_coordinates[0]- i[0]) <=300  and abs(selected_coordinates[1]- i[1]) <= 150:
                    green_rectangle_coordinates.remove(i)
                    green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                    break
                

                    


            image = cv2.rectangle(image, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
            image = cv2.putText(image, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        # if len(green_rectangle_coordinates) < 9:
        #     green_rectangle_coordinates.append([random.randint(0, 1593), random.randint(0, 873)])
        black_window.fill(0)

        for [x, y] in green_rectangle_coordinates:
            cv2.rectangle(black_window, (x, y), (x+200, y+100), (0, 255, 0), cv2.FILLED)

        
        
        
    elif touching_rectangle==False and snake_game==False:
        # Only draw the rectangle if touching_rectangle is False
        cv2.rectangle(image, (x1, y1), (x2, y2), (105, 136, 207), cv2.FILLED) 
        cv2.putText(image, f"Start Rectangle Game", (x1+10, y1+50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        cv2.rectangle(image, (x1, 50), (x2, 150), (105, 136, 207), cv2.FILLED) 
        cv2.putText(image, f"Start Snake Game", (x1+10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        cv2.rectangle(image, (x1, y2+30), (x2, y2+130), (105, 136, 207), cv2.FILLED) 
        cv2.putText(image, f"Start Vision Game", (x1+10, y2+30+50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        


    print(green_rectangle_coordinates)


    # if eye_control == False:  # Display the points counter  
    cv2.imshow("Games", image)
    if cv2.waitKey(5) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()


[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3], [3, 293], [533, 293]]
[[3, 3], [533, 3

In [4]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
# this model have accuracy of 100% and loss of 0.0015
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 5


# Initialize the webcam
eyes_detected = False

x = 0


for i in folders:
    
    path = f'photos/{i}'
    filenames = os.listdir(path)

    for files in filenames:
        path = f'photos/{i}/{files}'
    
        frame = cv2.imread(path)

        
        eyes = detect_eyes(frame)


        if len(eyes) >= 2:
            (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]

            eye_x = min(eye_1x, eye_2x)
            eye_y = min(eye_1y, eye_2y)
            # eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
            # eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)

            if eye_x == eye_1x:
                left_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
                right_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
            else:
                left_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
                right_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
            
            # Resize the ROI to match the desired dimensions
            resized_left_eye_roi = cv2.resize(left_eye_roi, (125, 100))
            resized_right_eye_roi = cv2.resize(right_eye_roi, (125, 100))
            
            combined_image = cv2.hconcat([resized_left_eye_roi, resized_right_eye_roi])



        

            x += 1
            if not os.path.exists(f'two_eyes/{i}'):
                os.makedirs(f'two_eyes/{i}')

            cv2.imwrite(f'two_eyes/{i}/{x}.jpg', combined_image)

    # cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# # Release the webcam and close the windows
# video_capture.release()
# cv2.destroyAllWindows()


NameError: name 'video_capture' is not defined

In [8]:
import pyautogui

# Set the desired coordinates
x = 500
y = 300

# Move the mouse cursor to the specified coordinates
pyautogui.moveTo(x, y)
